In [1]:
# Useful additional packages 
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from math import pi


from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import BasicAer, execute
from qiskit.quantum_info import Pauli, state_fidelity, basis_state, process_fidelity 

from qiskit.circuit import Parameter


from qiskit import *
IBMQ.load_accounts(hub=None)
from qiskit.tools.monitor import job_monitor, backend_monitor, backend_overview
from qiskit.providers.ibmq import least_busy

shots = 8192 
max_credits = 3 
#job_exp = execute(qc, backend=backend, shots=shots, max_credits=max_credits)
from qiskit.compiler import transpile

In [2]:
#Run in real quantum computer
#Misura di tutti i qubit
n = 6
q = QuantumRegister(n, 'q')
c = ClassicalRegister(n, 'c') #n

circ = QuantumCircuit(q,c)
phi = Parameter('phi')

circ.h(q[0])

for i in range(n-1):
    circ.cx(i, i+1)
    
circ.barrier()
circ.rz(phi, range(n))
circ.barrier()

for i in reversed(range(n-1)):
    circ.cx(i, i+1)

circ.h(q[0])

circ.barrier()

for i in range(n):
    circ.measure(q[i], c[i]) 

#circ.draw(output='mpl')

print('Phi range')
phi_range = [np.pi*j/(n+1) for j in range(2*n+1+1)]
print(phi_range)
circuits = [circ.bind_parameters({phi: phi_val}) for phi_val in phi_range] 
#print(circuits[3].parameters)
#circuits[11].draw(output='mpl')


job = execute(circ,
            backend=IBMQ.get_backend('ibmq_16_melbourne'),
            parameter_binds=[{phi: phi_val}
              for phi_val in phi_range],shots=shots)

job_monitor(job)

phi_range2 = phi_range
counts2 =[job.result().get_counts(i) for i in range(len(job.result().results))] 

print('\n')
print('Counts')
print(counts2)
print('\n')
print('Counts of 000000')
print(list(map(lambda c: 1/shots*c.get('000000', 0), counts2)))
job_id = job.job_id()
print('JOB ID: {}'.format(job_id))

Phi range
[0.0, 0.4487989505128276, 0.8975979010256552, 1.3463968515384828, 1.7951958020513104, 2.243994752564138, 2.6927937030769655, 3.141592653589793, 3.5903916041026207, 4.039190554615448, 4.487989505128276, 4.936788455641103, 5.385587406153931, 5.8343863566667595]
Job Status: job has successfully run


Counts
[{'010001': 240, '110110': 61, '100011': 84, '001101': 113, '110010': 99, '010101': 158, '010110': 82, '101001': 153, '101110': 32, '010000': 472, '010100': 235, '100001': 194, '000100': 203, '000110': 60, '111100': 94, '101100': 76, '111000': 226, '001011': 36, '101000': 183, '111011': 41, '100110': 49, '111001': 135, '001100': 117, '110111': 36, '001111': 28, '100111': 72, '011111': 36, '110101': 157, '000011': 107, '101011': 43, '100100': 180, '001110': 38, '100010': 87, '110011': 78, '111010': 58, '011101': 100, '100000': 388, '111111': 30, '000101': 160, '010111': 58, '111110': 37, '101101': 74, '110000': 399, '011011': 52, '011110': 47, '001000': 202, '101010': 43, '010

In [3]:
#Run in real quantum computer - Transpile
#Misura di tutti i qubit
n = 6
q = QuantumRegister(n, 'q')
c = ClassicalRegister(n, 'c') #n

circ = QuantumCircuit(q,c)
phi = Parameter('phi')

circ.h(q[0])

for i in range(n-1):
    circ.cx(i, i+1)
    
circ.barrier()
circ.rz(phi, range(n))
circ.barrier()

for i in reversed(range(n-1)):
    circ.cx(i, i+1)

circ.h(q[0])

circ.barrier()

for i in range(n):
    circ.measure(q[i], c[i]) 

#circ.draw(output='mpl')

print('Phi range')
phi_range = [np.pi*j/(n+1) for j in range(2*n+1+1)]
print(phi_range)
circuits = [circ.bind_parameters({phi: phi_val}) for phi_val in phi_range] 
#print(circuits[3].parameters)
#circuits[11].draw(output='mpl')

backend = IBMQ.get_backend('ibmq_16_melbourne')
trans_ghz = transpile(circ, backend)

job = execute(circ,
            backend,
            parameter_binds=[{phi: phi_val}
              for phi_val in phi_range],shots=shots)

job_monitor(job)

phi_range2 = phi_range
counts2 =[job.result().get_counts(i) for i in range(len(job.result().results))] 

print('\n')
print('Counts')
print(counts2)
print('\n')
print('Counts of 000000')
print(list(map(lambda c: 1/shots*c.get('000000', 0), counts2)))

job_id = job.job_id()
print('JOB ID: {}'.format(job_id))

#retrieved_job = backend.retrieve_job(job_id)
#retrieved_job.result().get_counts(qc)

Phi range
[0.0, 0.4487989505128276, 0.8975979010256552, 1.3463968515384828, 1.7951958020513104, 2.243994752564138, 2.6927937030769655, 3.141592653589793, 3.5903916041026207, 4.039190554615448, 4.487989505128276, 4.936788455641103, 5.385587406153931, 5.8343863566667595]
Job Status: job has successfully run


Counts
[{'010001': 285, '110110': 65, '100011': 78, '001101': 78, '110010': 123, '010101': 131, '010110': 62, '101001': 137, '101110': 35, '010000': 448, '010100': 177, '100001': 260, '000100': 164, '000110': 67, '111100': 97, '101100': 80, '111000': 255, '001011': 50, '101000': 213, '111011': 69, '100110': 56, '111001': 145, '001100': 88, '110111': 45, '001111': 31, '100111': 47, '011111': 31, '110101': 116, '000011': 95, '101011': 42, '100100': 138, '001110': 27, '100010': 137, '110011': 103, '111010': 86, '011101': 99, '100000': 362, '111111': 32, '000101': 122, '010111': 36, '111110': 40, '101101': 84, '110000': 378, '011011': 62, '011110': 35, '001000': 235, '101010': 46, '0100

In [6]:
#Run in real quantum computer - Transpile user
#Misura di tutti i qubit
n = 6
q = QuantumRegister(n, 'q')
c = ClassicalRegister(n, 'c') #n

circ = QuantumCircuit(q,c)
phi = Parameter('phi')

circ.h(q[0])

for i in range(n-1):
    circ.cx(i, i+1)
    
circ.barrier()
circ.rz(phi, range(n))
circ.barrier()

for i in reversed(range(n-1)):
    circ.cx(i, i+1)

circ.h(q[0])

circ.barrier()

for i in range(n):
    circ.measure(q[i], c[i]) 

#circ.draw(output='mpl')

print('Phi range')
phi_range = [np.pi*j/(n+1) for j in range(2*n+1+1)]
print(phi_range)
circuits = [circ.bind_parameters({phi: phi_val}) for phi_val in phi_range] 
#print(circuits[3].parameters)
#circuits[11].draw(output='mpl')

layout = {q[0]: 13, q[1]: 12, q[2]: 2, q[3]:3, q[4]:11, q[5]:10}

backend = IBMQ.get_backend('ibmq_16_melbourne')
trans_ghz_mapped = transpile(circ, backend, initial_layout=layout)

job = execute(circ,
            backend,
            parameter_binds=[{phi: phi_val}
              for phi_val in phi_range],shots=shots)

job_monitor(job)

phi_range2 = phi_range
counts2 =[job.result().get_counts(i) for i in range(len(job.result().results))] 

print('\n')
print('Counts')
print(counts2)
print('\n')
print('Counts of 000000')
print(list(map(lambda c: 1/shots*c.get('000000', 0), counts2)))

job_id = job.job_id()
print('JOB ID: {}'.format(job_id))

#retrieved_job = backend.retrieve_job(job_id)
#retrieved_job.result().get_counts(qc)

Phi range
[0.0, 0.4487989505128276, 0.8975979010256552, 1.3463968515384828, 1.7951958020513104, 2.243994752564138, 2.6927937030769655, 3.141592653589793, 3.5903916041026207, 4.039190554615448, 4.487989505128276, 4.936788455641103, 5.385587406153931, 5.8343863566667595]
Job Status: job has successfully run


Counts
[{'010001': 452, '110110': 29, '100011': 99, '001101': 39, '110010': 111, '010101': 82, '010110': 44, '101001': 162, '101110': 15, '010000': 518, '010100': 84, '100001': 384, '000100': 91, '000110': 39, '111100': 51, '101100': 53, '111000': 236, '001011': 66, '101000': 201, '111011': 54, '100110': 27, '111001': 146, '001100': 70, '110111': 24, '001111': 27, '100111': 36, '011111': 22, '110101': 67, '000011': 144, '101011': 65, '100100': 78, '001110': 28, '100010': 116, '110011': 100, '111010': 62, '011101': 64, '100000': 459, '111111': 28, '000101': 65, '010111': 43, '111110': 21, '101101': 45, '110000': 413, '011011': 78, '011110': 25, '001000': 247, '101010': 66, '010011': 

In [5]:
n = 6
q = QuantumRegister(n, 'q')
c = ClassicalRegister(n, 'c') #n

circ = QuantumCircuit(q,c)
phi = Parameter('phi')

circ.h(q[0])

for i in range(n-1):
    circ.cx(i, i+1)
    
circ.barrier()
circ.rz(phi, range(n))
circ.barrier()

for i in reversed(range(n-1)):
    circ.cx(i, i+1)

circ.h(q[0])

circ.barrier()

for i in range(n):
    circ.measure(q[i], c[i]) 

#circ.draw(output='mpl')

print('Phi range')
phi_range = [np.pi*j/(n+1) for j in range(2*n+1+1)]
print(phi_range)
circuits = [circ.bind_parameters({phi: phi_val}) for phi_val in phi_range] 
#print(circuits[3].parameters)
#circuits[11].draw(output='mpl')

layout = {q[0]: 13, q[1]: 12, q[2]: 2, q[3]:3, q[4]:11, q[5]:10}

backend = IBMQ.get_backend('ibmq_16_melbourne')
trans_ghz_mapped = transpile(circ, backend, initial_layout=layout)

trans_ghz = transpile(circ, backend)
print(circ.depth())
print(trans_ghz.depth())
print(trans_ghz_mapped.depth())

Phi range
[0.0, 0.4487989505128276, 0.8975979010256552, 1.3463968515384828, 1.7951958020513104, 2.243994752564138, 2.6927937030769655, 3.141592653589793, 3.5903916041026207, 4.039190554615448, 4.487989505128276, 4.936788455641103, 5.385587406153931, 5.8343863566667595]
14
20
18


In [2]:
#Run in real quantum computer - Transpile user NUOVO CIRCUITOO
#Misura di tutti i qubit
n = 6
q = QuantumRegister(n, 'q')
c = ClassicalRegister(n, 'c') #n

circ = QuantumCircuit(q,c)
phi = Parameter('phi')

circ.h(q[2])
circ.cx(q[2], q[0])
circ.cx(q[1], q[0])
circ.cx(q[2], q[3])
circ.cx(q[3], q[4])
circ.cx(q[4], q[5])
    
circ.barrier()
circ.rz(phi, range(n))
circ.barrier()

circ.cx(q[4], q[5])
circ.cx(q[3], q[4])
circ.cx(q[2], q[3])
circ.cx(q[1], q[0])
circ.cx(q[2], q[0])
circ.h(q[2])

circ.barrier()

for i in range(n):
    circ.measure(q[i], c[i]) 

#circ.draw(output='mpl')

print('Phi range')
phi_range = [np.pi*j/(n+1) for j in range(2*n+1+1)]
print(phi_range)
circuits = [circ.bind_parameters({phi: phi_val}) for phi_val in phi_range] 
#print(circuits[3].parameters)
#circuits[11].draw(output='mpl')

layout = {q[0]:0, q[1]: 1, q[2]: 13, q[3]:12, q[4]:2, q[5]:3}

backend = IBMQ.get_backend('ibmq_16_melbourne')
trans_ghz_mapped = transpile(circ, backend, initial_layout=layout)

job = execute(circ,
            backend,
            parameter_binds=[{phi: phi_val}
              for phi_val in phi_range],shots=shots)

job_monitor(job)

phi_range2 = phi_range
counts2 =[job.result().get_counts(i) for i in range(len(job.result().results))] 

print('\n')
print('Counts')
print(counts2)
print('\n')
print('Counts of 000000')
print(list(map(lambda c: 1/shots*c.get('000000', 0), counts2)))

job_id = job.job_id()
print('JOB ID: {}'.format(job_id))

#retrieved_job = backend.retrieve_job(job_id)
#retrieved_job.result().get_counts(qc)

Phi range
[0.0, 0.4487989505128276, 0.8975979010256552, 1.3463968515384828, 1.7951958020513104, 2.243994752564138, 2.6927937030769655, 3.141592653589793, 3.5903916041026207, 4.039190554615448, 4.487989505128276, 4.936788455641103, 5.385587406153931, 5.8343863566667595]
Job Status: job has successfully run


Counts
[{'010110': 73, '011011': 44, '000001': 304, '100010': 53, '110100': 137, '011111': 40, '110010': 25, '100110': 38, '100001': 80, '100111': 21, '101001': 82, '011101': 139, '100101': 86, '111100': 118, '101110': 36, '011001': 189, '000100': 490, '000000': 648, '101000': 109, '101101': 68, '000110': 123, '110110': 38, '001011': 86, '001101': 249, '101010': 30, '111001': 80, '001001': 256, '010101': 137, '011000': 322, '010010': 96, '011100': 200, '101011': 27, '000101': 223, '111011': 23, '100000': 132, '100011': 30, '110101': 59, '011110': 51, '001000': 586, '111101': 52, '100100': 181, '110001': 60, '001110': 91, '001111': 60, '010001': 213, '010000': 308, '000111': 73, '001